In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [ ]:
import matplotlib.pyplot as plt
import json

from PIL import Image
import cv2
import seaborn as sb

# Seb 06-01-21
import tensorflow as tf
from tensorflow import keras
import json
import math, re, os
from math import sqrt

from kaggle_datasets import KaggleDatasets
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from functools import partial



In [ ]:
main_dir = '../input/cassava-leaf-disease-classification/'
os.listdir(main_dir) 
train_img_path = '../input/cassava-leaf-disease-classification/train_images'

In [ ]:
#Detect TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
# set up variables
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')

GCS_NewTFPure512 = KaggleDatasets().get_gcs_path('cassava-leaf-disease-tfrecords-512x512')
GCS_NewTFCenter512 = KaggleDatasets().get_gcs_path('cassava-leaf-disease-tfrecords-center-512x512')
                                                 
IMAGE_SIZE = [512,512]
CLASSES = ['0', '1', '2', '3', '4']


In [ ]:
#Model variables
nets = 5
model = [0] * nets
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
EPOCHS = 70
input_shape = (512,512,3)
dropout_rate = 0.5

lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5, 
    decay_steps=10000, 
    decay_rate=0.90)

In [ ]:
filenames = [[0,0],[0,0],[0,0],[0,0],[0,0]]
seeds = [1,2,105,103,108]

for split in range(nets):
    filenames[split][0], filenames[split][1]= train_test_split(
    tf.io.gfile.glob(GCS_NewTFPure512+ '/Id_train*.tfrec'),
    test_size=0.3, random_state = seeds[split])

TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test00-1.tfrec')

filenames[0][1], filenames[1][1], filenames[2][1], filenames[3][1], filenames[4][1]

# Functions for datasets

In [ ]:
#Decode the data
#turn the images into tensors
#normalize the image (get every pixel to have a value between 0 and 1)
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [ ]:
#setting up variables X and y; in this case image and prediction (for images with no label)
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

In [ ]:
# the following code will load the dataset using the TPU
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
#The following functions will be used to load our training, validation, and test datasets, as well as print out the number of images in each dataset.
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)  
    dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)  
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered) 
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)



# NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
# NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)


# print('Dataset: {} training images, {} validation images, {} (unlabeled) test images'.format(
#     NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:

#adding in augmentations
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
#    image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_contrast(image)
#     image = tf.image.random_jpeg_quality(image)
    return image, label

In [ ]:
# # # load data
# train_dataset = get_training_dataset()
# valid_dataset = get_validation_dataset()


In [ ]:
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
# VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE

# Model

In [ ]:
# from keras.optimizers import RMSprop, Adam
# from keras.layers import DepthwiseConv2D, Reshape, Activation
# import efficientnet.keras as efn
# from tensorflow.keras import models
# from tensorflow.keras import layers

# for j in range(nets):
#     with strategy.scope(): 
#         conv_base = efn.EfficientNetB3(weights='imagenet', include_top = False, input_shape = input_shape)
#         model[j] = models.Sequential()
#         model[j].add(conv_base)
#         model[j].add(layers.GlobalMaxPooling2D(name="gap"))
#         model[j].add(layers.Dropout(dropout_rate, name="dropout_out"))
#         model[j].add(layers.Dense(5, activation="softmax", name="fc_out"))
#         model[j].compile(
#             optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])  

In [ ]:
# for i in range(nets):
#     model[i].summary()

In [ ]:
# train_set = [0,0,0,0,0]
# valid_set = [0,0,0,0,0]

# for i in range(nets):
#     TRAINING_FILENAMES = filenames[i][0]
#     VALID_FILENAMES = filenames[i][1]
    
#     train_set[i] = get_training_dataset()
#     valid_set[i] = get_validation_dataset()
    
    
# print(train_set)
# print(valid_set)

In [ ]:
# # train for 20 epochs
# history = [0] * nets
# EPOCHS = 70

# for j in range(nets):
#     STEPS_PER_EPOCH = count_data_items(filenames[i][0]) // BATCH_SIZE
#     VALID_STEPS = count_data_items(filenames[i][1]) // BATCH_SIZE
#     print(f'Individual Net : {j+1}')   
#     history[j] = model[j].fit(train_set[j], 
#                     steps_per_epoch=STEPS_PER_EPOCH, 
#                     epochs=EPOCHS,
#                     validation_data=valid_set[j],
#                     validation_steps=VALID_STEPS)

#     print("CNN Model {0:d}: Epochs={1:d}, Training accuracy={2:.5f}, Validation accuracy={3:.5f}".format(j+1,EPOCHS,max(history[j].history['sparse_categorical_accuracy']),max(history[j].history['val_sparse_categorical_accuracy']) ))

In [ ]:
# for index in range (nets):
#     model[index].save('model{}.h5'.format(index))

# Predictions 

In [ ]:
# def to_float32(image, label):
#     return tf.cast(image, tf.float32), label

# test_dataset = get_test_dataset()

# test_ds = get_test_dataset(ordered=True) 
# test_ds = test_ds.map(to_float32)

# test_images_ds = test_dataset
# test_images_ds = test_ds.map(lambda image, idnum: image)

In [ ]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)

# #Result
# results = np.zeros( (int(NUM_TEST_IMAGES),5) ) 
# for j in range(nets):
#     #each model gives its predictions
#     results = results + model[j].predict(test_images_ds)
 
# #Finding the best prediction
# predictions = np.argmax(results,axis = 1)

In [ ]:
# import matplotlib.pyplot as plt
# test_dataset = get_test_dataset()


# print('done')

# for element in test_dataset.as_numpy_iterator(): 
#     plt.imshow(element[0][0])
#     plt.title(predictions[0])
#     print(element[1])

In [ ]:
# #Printing some images with predictions as titles - used only for visualization

# L = 4
# W = 4

# fig, axes = plt.subplots(L, W, figsize = (12,12))
# axes = axes.ravel()

# for element in test_dataset.as_numpy_iterator(): 
#     for i in np.arange(0, L * W):  
#         axes[i].imshow(element[0][i])
#         axes[i].set_title(predictions[i])
#         axes[i].axis('off')

# plt.subplots_adjust(wspace=0.5)


# Everything down below is not important

In [ ]:
# import efficientnet.keras as efn
# from tensorflow.keras import models
# from tensorflow.keras import layers

# #model
# # with strategy.scope():       
# # #    img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
    
# # #     base_model = efn.EfficientNetB3(weights='imagenet')
# #     conv_base = efn.EfficientNetB3(weights='imagenet', include_top = False, input_shape = input_shape)
# #     model = models.Sequential()
# #     model.add(conv_base)
# #     model.add(layers.GlobalMaxPooling2D(name="gap"))
# #     model.add(layers.Dropout(dropout_rate, name="dropout_out"))
# #     model.add(layers.Dense(5, activation="softmax", name="fc_out"))

# def evaluate_model(trainX, testX):
# #     trainy_enc = to_categorical(trainy)
# #     testy_enc = to_categorical(testy)
#     # define model
#     with strategy.scope():           
#             conv_base = efn.EfficientNetB3(weights='imagenet', include_top = False, input_shape = input_shape)
#             model = models.Sequential()
#             model.add(conv_base)
#             model.add(layers.GlobalMaxPooling2D(name="gap"))
#             model.add(layers.Dropout(dropout_rate, name="dropout_out"))
#             model.add(layers.Dense(5, activation="softmax", name="fc_out"))
#             model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])    

#     #trainX is train_dataset; trainy_enc is valid_dataset
#     model.fit(trainX, epochs=EPOCHS, verbose=0, steps_per_epoch=STEPS_PER_EPOCH)
#     _, test_acc = model.evaluate(testX, verbose=0)
#     return model, test_acc

In [ ]:
#Because we're using a pre-trained model, we expect there to be a large number of non-trainable parameters (because the weights have already been assigned in the pre-trained model).

# model.summary()

In [ ]:
# #Evaluating our model
# # print out variables available to us
# print(history.history.keys())


# # create learning curves to evaluate model performance
# history_frame = pd.DataFrame(history.history)
# history_frame.loc[:, ['loss', 'val_loss']].plot()
# history_frame.loc[:, ['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy']].plot();

In [ ]:
# def to_float32(image, label):
#     return tf.cast(image, tf.float32), label

# testing_dataset = get_test_dataset()
# test_ds = get_test_dataset(ordered=True) 
# test_ds = test_ds.map(to_float32)

# print('Computing predictions...')
# test_images_ds = testing_dataset
# test_images_ds = test_ds.map(lambda image, idnum: image)
# probabilities = model.predict(test_images_ds)
# predictions = np.argmax(probabilities, axis=-1)
# print(predictions)

In [ ]:
# def dataset_to_numpy_util(dataset, N):
#     dataset = dataset.unbatch().batch(N)
#     for images, labels in dataset:
#         numpy_images = images.numpy()
#         numpy_labels = labels.numpy()
#         break;  
#     return numpy_images, numpy_labels

# def title_from_label_and_target(label, correct_label):
#     label = np.argmax(label, axis=-1)
#     correct = (label == correct_label)
#     return "{} [{}{}{}]".format(label, str(correct), ', shoud be ' if not correct else '',
#                                 correct_label if not correct else ''), correct

# def display_one_flower_eval(image, title, subplot, red=False):
#     plt.subplot(subplot)
#     plt.axis('off')
#     plt.imshow(image)
#     plt.title(title, fontsize=14, color='red' if red else 'black')
#     return subplot+1

# def display_9_images_with_predictions(images, predictions, labels):
#     subplot=331
#     plt.figure(figsize=(13,13))
#     for i, image in enumerate(images):
#         title, correct = title_from_label_and_target(predictions[i], labels[i])
#         subplot = display_one_flower_eval(image, title, subplot, not correct)
#         if i >= 8:
#             break;
              
#     plt.tight_layout()
#     plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.show()

In [ ]:
# x_samp, y_samp = dataset_to_numpy_util(testingg_dataset, 27)

# x_samp_1, y_samp_1 = x_samp[:9,:,:,:], y_samp[:9]
# samp_preds_1 = model.predict(x_samp_1, batch_size=9)
# display_9_images_with_predictions(x_samp_1, samp_preds_1, y_samp_1)

# x_samp_2, y_samp_2 = x_samp[9:,:,:,:], y_samp[9:]
# samp_preds_2 = model.predict(x_samp_2, batch_size=9)
# display_9_images_with_predictions(x_samp_2, samp_preds_2, y_samp_2)

In [ ]:
# def dataset_to_numpy(dataset):
#     for images, labels in dataset:
#         numpy_images = images.numpy()
#         numpy_labels = labels.numpy()
#         break;  
#     return numpy_images, numpy_labels

# def div(prediction, correct_labels, images):
#     totalCount = NUM_TESTING_IMAGES
#     hits = 0
    
#     for i, image in enumerate(images):
#         prediction[i] = np.argmax(prediction[i], axis=-1)
#         if ((prediction[i] == correct_labels[i]).any()):
#             hits+=1

# #     for index in range(0,totalCount-1):
# #         if prediction[index] == correct_labels[index]:
# #             hits+=1
#     return'The amount of correct predictions is {} out of {} which means that the model has an accuracy of {}%'.format(hits,totalCount,round((hits/totalCount)*100),2)

In [ ]:
# print(testingg_dataset)
# for images, labels in testingg_dataset:
#         print(images.numpy())
#         print(labels.numpy())

In [ ]:
# samp_preds = model.predict(x_samp)
# for predictions in samp_preds:
#     print(predictions)


In [ ]:
# #images = x_samp
# #predictions = samp_preds
# #labels = y_samp

# # x_samp img, y_samp labels, samp_preds predictions
# x_samp, y_samp = dataset_to_numpy(testingg_dataset)
# samp_preds = model.predict(x_samp)
# div(samp_preds,y_samp,x_samp)

In [ ]:
# import efficientnet.keras as efn
# from tensorflow.keras import models
# from tensorflow.keras import layers

# #model
# with strategy.scope():       
# #    img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
    
# #     base_model = efn.EfficientNetB3(weights='imagenet')
#     conv_base = efn.EfficientNetB3(weights='imagenet', include_top = False, input_shape = input_shape)
#     model = models.Sequential()
#     model.add(conv_base)
#     model.add(layers.GlobalMaxPooling2D(name="gap"))
#     model.add(layers.Dropout(dropout_rate, name="dropout_out"))
#     model.add(layers.Dense(5, activation="softmax", name="fc_out"))
    


#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),
#         loss='sparse_categorical_crossentropy',  
#         metrics=['sparse_categorical_accuracy'])
    
#     model.summary()

In [ ]:
# history = model.fit(train_dataset, 
#                     steps_per_epoch=STEPS_PER_EPOCH, 
#                     epochs=EPOCHS,
#                     validation_data=valid_dataset,
#                     validation_steps=VALID_STEPS)

In [ ]:
# import efficientnet.keras as efn
# from tensorflow.keras import models
# from tensorflow.keras import layers

# #model
# # with strategy.scope():       
# # #    img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
    
# # #     base_model = efn.EfficientNetB3(weights='imagenet')
# #     conv_base = efn.EfficientNetB3(weights='imagenet', include_top = False, input_shape = input_shape)
# #     model = models.Sequential()
# #     model.add(conv_base)
# #     model.add(layers.GlobalMaxPooling2D(name="gap"))
# #     model.add(layers.Dropout(dropout_rate, name="dropout_out"))
# #     model.add(layers.Dense(5, activation="softmax", name="fc_out"))

# def evaluate_model(trainX, testX):
# #     trainy_enc = to_categorical(trainy)
# #     testy_enc = to_categorical(testy)
#     # define model
#     with strategy.scope():           
#             conv_base = efn.EfficientNetB3(weights='imagenet', include_top = False, input_shape = input_shape)
#             model = models.Sequential()
#             model.add(conv_base)
#             model.add(layers.GlobalMaxPooling2D(name="gap"))
#             model.add(layers.Dropout(dropout_rate, name="dropout_out"))
#             model.add(layers.Dense(5, activation="softmax", name="fc_out"))
#             model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])    

#     #trainX is train_dataset; trainy_enc is valid_dataset
#     model.fit(trainX, epochs=EPOCHS, verbose=0, steps_per_epoch=STEPS_PER_EPOCH)
#     _, test_acc = model.evaluate(testX, verbose=0)
#     return model, test_acc

In [ ]:
# def ensemble_predictions(members, testX):
#     test_images_ds = testX
#     test_images_ds = test_ds.map(lambda image, idnum: image)
#     # make predictions
#     yhats = [model.predict(test_images_ds) for model in members]
#     yhats = array(yhats)
#     # sum across ensemble members
#     summed = numpy.sum(yhats, axis=0)
#     # argmax across classes
#     result = argmax(summed, axis=1)
#     return result

In [ ]:
# from sklearn.metrics import accuracy_score
# def evaluate_n_members(members, n_members, testX, testy):
# 	# select a subset of members
# 	subset = members[:n_members]
# 	# make prediction
# 	yhat = ensemble_predictions(subset, testX)
# 	# calculate accuracy
# 	return accuracy_score(testy, yhat)

In [ ]:
# from sklearn.datasets import make_blobs

# # generate 2d classification dataset
# dataX, datay = make_blobs(n_samples=1427, centers=3, n_features=4, cluster_std=2, random_state=2)
# X, newX = dataX[:5000, :], dataX[5000:, :]
# y, newy = datay[:5000], datay[:]

In [ ]:
# from matplotlib import pyplot
# from numpy import mean
# from numpy import std
# import numpy
# from numpy import array
# from numpy import argmax

# scores, members = list(), list()
# model, test_acc = evaluate_model(train_dataset, valid_dataset)

# print('>%.3f' % test_acc)
# scores.append(test_acc)
# members.append(model)
    
# print('Estimated Accuracy %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
# from keras.utils import to_categorical

# single_scores, ensemble_scores = list(), list()

# ensemble_score = evaluate_n_members(members, 1, testing_dataset, newy)
# newy_enc = to_categorical(newy)
# _, single_score = members[1-1].evaluate(newX, newy_enc, verbose=0)
# print('> %d: single=%.3f, ensemble=%.3f' % (1, single_score, ensemble_score))
# ensemble_scores.append(ensemble_score)
# single_scores.append(single_score)

In [ ]:
# print('Accuracy %.3f (%.3f)' % (mean(single_scores), std(single_scores)))
# x_axis = [i for i in range(1, 0+1)]
# pyplot.plot(x_axis, single_scores, marker='o', linestyle='None')
# pyplot.plot(x_axis, ensemble_scores, marker='o')
# pyplot.show()